# FLUX.2 Klein 4B Character LoRA Pipeline

mflux로 Mac에서 학습 → HuggingFace 업로드 → Runware AI 추론

## Prerequisites
- mflux >= 0.16.5 (`pip install mflux`)
- `.env` file with HF_TOKEN, RUNWARE_API_KEY, HF_REPO_ID

In [ ]:
import os, sys, json, shutil, zipfile, subprocess
from pathlib import Path
from dotenv import load_dotenv

# Project root
ROOT = Path(os.getcwd()).parent if Path(os.getcwd()).name == "notebooks" else Path(os.getcwd())
os.chdir(ROOT)
sys.path.insert(0, str(ROOT / "scripts"))

# Load environment
load_dotenv()
HF_TOKEN = os.getenv("HF_TOKEN")
RUNWARE_API_KEY = os.getenv("RUNWARE_API_KEY")
HF_REPO_ID = os.getenv("HF_REPO_ID")

assert HF_TOKEN, "HF_TOKEN not set in .env"
assert RUNWARE_API_KEY, "RUNWARE_API_KEY not set in .env"
assert HF_REPO_ID, "HF_REPO_ID not set in .env"
print(f"Project root: {ROOT}")
print(f"HF repo: {HF_REPO_ID}")
print("API keys loaded OK")

## 1. Configuration

Set your character name and trigger word here. All subsequent cells use these values.

In [ ]:
# === EDIT THESE ===
CHARACTER_NAME = "my_character"       # directory name, no spaces
TRIGGER_WORD = "ohwx_mychar"          # unique trigger word
LORA_VERSION = "1"                    # increment for new versions

# === Derived paths (don't edit) ===
RAW_DIR = ROOT / f"datasets/{CHARACTER_NAME}/raw"
PROCESSED_DIR = ROOT / f"datasets/{CHARACTER_NAME}/processed"
CAPTIONS_DIR = ROOT / f"datasets/{CHARACTER_NAME}/captions"
MFLUX_DATA_DIR = ROOT / f"datasets/{CHARACTER_NAME}/mflux"
TRAIN_CONFIG = ROOT / "config/train.json"
CHECKPOINTS_DIR = ROOT / "models/checkpoints"

for d in [RAW_DIR, PROCESSED_DIR, CAPTIONS_DIR, MFLUX_DATA_DIR, CHECKPOINTS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print(f"Character: {CHARACTER_NAME}")
print(f"Trigger: {TRIGGER_WORD}")
print(f"Raw images go in: {RAW_DIR}")

## 2. Data Preparation

1. Place 15-25 raw images in the `raw/` directory shown above
2. Run preprocessing (resize to 1024x1024 PNG)
3. Run captioning (Florence-2 + trigger word)
4. Merge into mflux data directory (image + txt pairs in same folder)

In [ ]:
from preprocess_images import preprocess_directory

results = preprocess_directory(RAW_DIR, PROCESSED_DIR, size=1024, prefix="char")
print(f"\nProcessed {len(results)} images")
if not results:
    print("\u26a0 No images found. Place images in:", RAW_DIR)

In [ ]:
from caption_images import caption_directory

results = caption_directory(
    PROCESSED_DIR,
    CAPTIONS_DIR,
    trigger_word=TRIGGER_WORD,
    device="mps",  # Apple Silicon
)
print(f"\nGenerated {len(results)} captions")

In [ ]:
# mflux expects image + txt pairs in the same directory
# Copy processed images and captions into mflux data dir

import shutil

# Clean previous data
for f in MFLUX_DATA_DIR.iterdir():
    f.unlink()

count = 0
for img_path in sorted(PROCESSED_DIR.glob("*.png")):
    txt_path = CAPTIONS_DIR / f"{img_path.stem}.txt"
    if not txt_path.exists():
        print(f"WARNING: no caption for {img_path.name}, skipping")
        continue
    shutil.copy2(img_path, MFLUX_DATA_DIR / img_path.name)
    shutil.copy2(txt_path, MFLUX_DATA_DIR / txt_path.name)
    count += 1

# Add preview prompt for monitoring training progress
preview_path = MFLUX_DATA_DIR / "preview_1.txt"
preview_path.write_text(f"{TRIGGER_WORD}, front view, neutral expression, white background, flat illustration style")

print(f"Prepared {count} image+caption pairs in {MFLUX_DATA_DIR}")
print(f"Preview prompt: {preview_path.read_text()}")

## 3. Training

Runs mflux-train with the config template. Updates the data path to point to our character's mflux directory.

In [ ]:
# Update train.json with current character's data path
config = json.loads(TRAIN_CONFIG.read_text())
config["data"] = str(MFLUX_DATA_DIR) + "/"
TRAIN_CONFIG.write_text(json.dumps(config, indent=2))

print(f"Training config updated: data = {config['data']}")
print(f"Model: {config['model']}")
print(f"Epochs: {config['training_loop']['num_epochs']}")
print(f"Batch size: {config['training_loop']['batch_size']}")
print(f"\nStarting training... (this will take a while)")

# Run training
result = subprocess.run(
    ["mflux-train", "--config", str(TRAIN_CONFIG)],
    cwd=str(ROOT),
)

if result.returncode == 0:
    print("\nTraining complete!")
else:
    print(f"\nTraining failed with exit code {result.returncode}")

## 4. Extract LoRA from Checkpoint

mflux saves checkpoints as ZIP files. We extract the safetensors adapter from the best checkpoint.

In [ ]:
# Find the latest training run
train_dirs = sorted(ROOT.glob("train_*"), key=lambda p: p.name)
if not train_dirs:
    raise FileNotFoundError("No training output found. Run training first.")

latest_run = train_dirs[-1]
checkpoint_dir = latest_run / "checkpoints"
checkpoints = sorted(checkpoint_dir.glob("*_checkpoint.zip"))

print(f"Training run: {latest_run.name}")
print(f"Found {len(checkpoints)} checkpoints:")
for cp in checkpoints:
    print(f"  {cp.name}")

# Use the latest checkpoint (highest step count)
best_checkpoint = checkpoints[-1]
print(f"\nUsing: {best_checkpoint.name}")

# Extract safetensors adapter
output_safetensors = CHECKPOINTS_DIR / f"{CHARACTER_NAME}.safetensors"
with zipfile.ZipFile(best_checkpoint) as zf:
    adapter_files = [f for f in zf.namelist() if f.endswith("_adapter.safetensors")]
    if not adapter_files:
        raise FileNotFoundError(f"No adapter.safetensors found in {best_checkpoint.name}")

    adapter_name = adapter_files[0]
    with zf.open(adapter_name) as src, open(output_safetensors, "wb") as dst:
        dst.write(src.read())

size_mb = output_safetensors.stat().st_size / (1024 * 1024)
print(f"\nExtracted: {output_safetensors}")
print(f"Size: {size_mb:.1f} MB")

## 5. Local Test

Quick inference with mflux-generate to verify the LoRA works before uploading.

In [ ]:
test_prompt = f"{TRIGGER_WORD}, front view, neutral expression, white background, flat illustration style, clean linework"
test_output = ROOT / "validation/results" / f"{CHARACTER_NAME}_test.png"

result = subprocess.run([
    "mflux-generate",
    "--prompt", test_prompt,
    "--model", "flux2-klein-base-4b",
    "--steps", "20",
    "--seed", "42",
    "--lora-paths", str(output_safetensors),
    "--lora-scales", "0.8",
    "--output", str(test_output),
])

if result.returncode == 0:
    from PIL import Image
    img = Image.open(test_output)
    display(img)
    print(f"Saved: {test_output}")
else:
    print("Generation failed. Check mflux installation.")

## 6. Upload to HuggingFace Hub

Uploads the safetensors file to a public HF repo so Runware can download it.

In [ ]:
from huggingface_hub import HfApi

api = HfApi(token=HF_TOKEN)

# Create repo if it doesn't exist
api.create_repo(repo_id=HF_REPO_ID, repo_type="model", private=False, exist_ok=True)

# Upload safetensors
hf_filename = f"{CHARACTER_NAME}.safetensors"
api.upload_file(
    path_or_fileobj=str(output_safetensors),
    path_in_repo=hf_filename,
    repo_id=HF_REPO_ID,
    repo_type="model",
)

download_url = f"https://huggingface.co/{HF_REPO_ID}/resolve/main/{hf_filename}"
print(f"Uploaded: {hf_filename}")
print(f"Download URL: {download_url}")

## 7. Register on Runware AI

Registers the LoRA with Runware so it can be used for inference via API.

**Note:** The `architecture` field uses `flux1d` as FLUX.2-specific values are not yet in the SDK. If registration fails, contact Runware support for the correct architecture value.

In [ ]:
import asyncio
from runware import Runware, IUploadModelLora

async def register_lora():
    runware = Runware(api_key=RUNWARE_API_KEY)
    await runware.connect()

    payload = IUploadModelLora(
        air=f"civitai:{CHARACTER_NAME}@{LORA_VERSION}",
        name=f"story-shorts-{CHARACTER_NAME}",
        downloadURL=download_url,
        uniqueIdentifier=f"story-shorts-{CHARACTER_NAME}-v{LORA_VERSION}",
        version=LORA_VERSION,
        architecture="flux1d",
        format="safetensors",
        positiveTriggerWords=TRIGGER_WORD,
        private=True,
        shortDescription=f"Character LoRA for {CHARACTER_NAME} (FLUX.2 Klein 4B)",
    )

    result = await runware.modelUpload(payload)
    return result

upload_result = asyncio.run(register_lora())
print(f"Runware registration result: {upload_result}")
LORA_AIR = f"civitai:{CHARACTER_NAME}@{LORA_VERSION}"
print(f"LoRA AIR ID: {LORA_AIR}")

## 8. Runware Inference Test

Generate images using the deployed LoRA via Runware API.

In [ ]:
from runware import Runware, IImageInference, ILora

async def run_inference(prompt: str, lora_weight: float = 0.8):
    runware = Runware(api_key=RUNWARE_API_KEY)
    await runware.connect()

    payload = IImageInference(
        positivePrompt=prompt,
        model="runware:400@5",  # FLUX.2 Klein 4B Base
        lora=[ILora(model=LORA_AIR, weight=lora_weight)],
        width=1024,
        height=1024,
        numberResults=1,
    )

    images = await runware.imageInference(requestImage=payload)
    return images

# Test prompts
test_prompts = [
    f"{TRIGGER_WORD}, front view, neutral expression, white background, flat illustration",
    f"{TRIGGER_WORD}, sitting and reading a book, cozy room, warm lighting, flat illustration",
    f"{TRIGGER_WORD}, walking down a city street, daytime, flat style",
]

from IPython.display import display, Image as IPImage
import urllib.request

for i, prompt in enumerate(test_prompts):
    print(f"\n[{i+1}/{len(test_prompts)}] {prompt[:60]}...")
    images = asyncio.run(run_inference(prompt))
    for img in images:
        print(f"  URL: {img.imageURL}")
        # Download and display
        img_path = ROOT / f"validation/results/{CHARACTER_NAME}_runware_{i+1}.png"
        urllib.request.urlretrieve(img.imageURL, str(img_path))
        display(IPImage(filename=str(img_path)))